# Multi-Class Prediction of Obesity Risk
Input files - [Downloaded here](https://www.kaggle.com/competitions/playground-series-s4e2/data)
1. *train.csv*
2. *test.csv*

## Development Notes
* Save DataFrame as pickle (to_pickle) to preserve column data types -- load with read_pickle

In [1]:
### libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Load and Preview Data

In [2]:
### load data
train_raw = pd.read_csv('Data Download/train.csv')
test_raw=pd.read_csv('Data Download/test.csv')

### data info
train_raw.info()
print("\n")
test_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20758 entries, 0 to 20757
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              20758 non-null  int64  
 1   Gender                          20758 non-null  object 
 2   Age                             20758 non-null  float64
 3   Height                          20758 non-null  float64
 4   Weight                          20758 non-null  float64
 5   family_history_with_overweight  20758 non-null  object 
 6   FAVC                            20758 non-null  object 
 7   FCVC                            20758 non-null  float64
 8   NCP                             20758 non-null  float64
 9   CAEC                            20758 non-null  object 
 10  SMOKE                           20758 non-null  object 
 11  CH2O                            20758 non-null  float64
 12  SCC                             

In [3]:
### preview data
train_raw.head(5)

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,0,Male,24.443011,1.699998,81.669950,yes,yes,2.000000,2.983297,Sometimes,no,2.763573,no,0.000000,0.976473,Sometimes,Public_Transportation,Overweight_Level_II
1,1,Female,18.000000,1.560000,57.000000,yes,yes,2.000000,3.000000,Frequently,no,2.000000,no,1.000000,1.000000,no,Automobile,Normal_Weight
2,2,Female,18.000000,1.711460,50.165754,yes,yes,1.880534,1.411685,Sometimes,no,1.910378,no,0.866045,1.673584,no,Public_Transportation,Insufficient_Weight
3,3,Female,20.952737,1.710730,131.274851,yes,yes,3.000000,3.000000,Sometimes,no,1.674061,no,1.467863,0.780199,Sometimes,Public_Transportation,Obesity_Type_III
4,4,Male,31.641081,1.914186,93.798055,yes,yes,2.679664,1.971472,Sometimes,no,1.979848,no,1.967973,0.931721,Sometimes,Public_Transportation,Overweight_Level_II


In [4]:
### preview data cont.
train_raw.describe(include='all')

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
count,20758.00000,20758,20758.000000,20758.000000,20758.000000,20758,20758,20758.000000,20758.000000,20758,20758,20758.000000,20758,20758.000000,20758.000000,20758,20758,20758
unique,NaN,2,NaN,NaN,NaN,2,2,NaN,NaN,4,2,NaN,2,NaN,NaN,3,5,7
top,NaN,Female,NaN,NaN,NaN,yes,yes,NaN,NaN,Sometimes,no,NaN,no,NaN,NaN,Sometimes,Public_Transportation,Obesity_Type_III
freq,NaN,10422,NaN,NaN,NaN,17014,18982,NaN,NaN,17529,20513,NaN,20071,NaN,NaN,15066,16687,4046
mean,10378.50000,NaN,23.841804,1.700245,87.887768,NaN,NaN,2.445908,2.761332,NaN,NaN,2.029418,NaN,0.981747,0.616756,NaN,NaN,NaN
std,5992.46278,NaN,5.688072,0.087312,26.379443,NaN,NaN,0.533218,0.705375,NaN,NaN,0.608467,NaN,0.838302,0.602113,NaN,NaN,NaN
min,0.00000,NaN,14.000000,1.450000,39.000000,NaN,NaN,1.000000,1.000000,NaN,NaN,1.000000,NaN,0.000000,0.000000,NaN,NaN,NaN
25%,5189.25000,NaN,20.000000,1.631856,66.000000,NaN,NaN,2.000000,3.000000,NaN,NaN,1.792022,NaN,0.008013,0.000000,NaN,NaN,NaN
50%,10378.50000,NaN,22.815416,1.700000,84.064875,NaN,NaN,2.393837,3.000000,NaN,NaN,2.000000,NaN,1.000000,0.573887,NaN,NaN,NaN
75%,15567.75000,NaN,26.000000,1.762887,111.600553,NaN,NaN,3.000000,3.000000,NaN,NaN,2.549617,NaN,1.587406,1.000000,NaN,NaN,NaN


## Pre-processing

In [5]:
## create new tables for manipulation
train_clean = train_raw.drop('id',axis=1)
test_clean = test_raw.drop('id',axis=1)

### Fill NULLS

In [6]:
## identify columns with NaN -- no NaNs
train_clean.columns[train_clean.isna().any()]
test_clean.columns[test_clean.isna().any()]

Index([], dtype='object')

### Fix Column DataTypes

In [7]:
## convert object columns to category
for col in train_clean.columns[train_clean.dtypes==object]:
    train_clean[col] = train_clean[col].astype('category')

for col in test_clean.columns[test_clean.dtypes==object]:
    test_clean[col] = test_clean[col].astype('category')

## display categories of category columns
for col in train_clean.columns[train_clean.dtypes=='category']:
    print(col, ': \t', train_clean[col].cat.categories)

Gender : 	 Index(['Female', 'Male'], dtype='object')
family_history_with_overweight : 	 Index(['no', 'yes'], dtype='object')
FAVC : 	 Index(['no', 'yes'], dtype='object')
CAEC : 	 Index(['Always', 'Frequently', 'Sometimes', 'no'], dtype='object')
SMOKE : 	 Index(['no', 'yes'], dtype='object')
SCC : 	 Index(['no', 'yes'], dtype='object')
CALC : 	 Index(['Frequently', 'Sometimes', 'no'], dtype='object')
MTRANS : 	 Index(['Automobile', 'Bike', 'Motorbike', 'Public_Transportation', 'Walking'], dtype='object')
NObeyesdad : 	 Index(['Insufficient_Weight', 'Normal_Weight', 'Obesity_Type_I',
       'Obesity_Type_II', 'Obesity_Type_III', 'Overweight_Level_I',
       'Overweight_Level_II'],
      dtype='object')


In [8]:
## set ordering for ordered categorical columns
for col in ['CAEC', 'CALC']:
    train_clean[col] = pd.Categorical(train_clean[col], categories=['no','Sometimes','Frequently','Always'], ordered=True)
    test_clean[col] = pd.Categorical(test_clean[col], categories=['no','Sometimes','Frequently','Always'], ordered=True)


train_clean['NObeyesdad'] = pd.Categorical(train_clean['NObeyesdad'], categories=['Insufficient_Weight', 'Normal_Weight', 'Overweight_Level_I', 'Overweight_Level_II', 
                                                                              'Obesity_Type_I', 'Obesity_Type_II', 'Obesity_Type_III'], ordered=True)

## Save data

In [9]:
train_clean.to_pickle('train_clean.pkl')
test_clean.to_pickle('test_clean.pkl')